# Data Preparation Exercises

## Imports

In [ ]:
# unicode, regex, json for text digestion
import unicodedata
import re
import json

# nltk: natural language toolkit -> tokenization, stopwords
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import acquire
from time import strftime

import warnings
warnings.filterwarnings('ignore')

## Exercises

The end result of this exercise should be a file named `prepare.py` that defines the requested functions.

In this exercise we will be defining some functions to prepare textual data. These functions should apply equally well to both the codeup blog articles and the news articles that were previously acquired.

### 1. Define a function named `basic_clean`. It should take in a string and apply some basic text cleaning to it:

* Lowercase everything
* Normalize unicode characters
* Replace anything that is not a letter, number, whitespace or a single quote.

In [ ]:
# This function takes in a string and returns the string normalized
def basic_clean(string):
    # we will normalize our data into standard NFKD unicode, feed it into an ascii encoding
    # decode it back into UTF-8
    string = unicodedata.normalize('NFKD', string)\
             .encode('ascii', 'ignore')\
             .decode('utf-8', 'ignore')
    # remove special characters, then lowercase
    string = re.sub(r"[^\w0-9'\s]", '', string).lower()
    return string

### 2. Define a function named `tokenize`. It should take in a string and tokenize all the words in the string.

In [ ]:
# This functions takes in a string and returns a tokenized string
def tokenize(string):
    # make our tokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()
    # apply our tokenization to the string input
    string = tokenizer.tokenize(string, return_str = True)
    
    return string

### 3. Define a function named `stem`. It should accept some text and return the text after applying stemming to all the words.

In [ ]:
# This function takes in a string and returns a string with words stemmed
def stem(string):
    # create our stemming object
    ps = nltk.porter.PorterStemmer()
    # use a list comprehension => stem each word inside of the entire document and split by single spaces
    stems = [ps.stem(word) for word in string.split()]
    # join it together with spaces
    string = ' '.join(stems)
    
    return string

### 4. Define a function named `lemmatize`. It should accept some text and return the text after applying lemmatization to each word.

In [ ]:
# This function takes in a string  and returns a string with words lemmatized
def lemmatize(string):
    # create our lemmatizer object
    wnl = nltk.stem.WordNetLemmatizer()
    # use a list comprehension to lemmatize each word
    # string.split() => output a list of every token inside of the document
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    # join the lemmas together with spaces
    string = ' '.join(lemmas)
    #return the altered document
    return string

### 5. Define a function named `remove_stopwords`. It should accept some text and return the text after removing all the stopwords. 

### This function should define two optional parameters, `extra_words` and `exclude_words`. These parameters should define any additional stop words to include, and any words that we don't want to remove.



In [ ]:
list1 = [1, 2, 3, 4]
list2 = [2, 1, 3, 4]

print(set(list1)==set(list2))

In [ ]:
list1 == list2

In [ ]:
mylist = ['a', 'b', 'c', 'c', 'd']

myset = set(mylist)

print(mylist, myset)

In [ ]:
# This function takes in a string, optional extra_words and exclued_words parameters with default empty lists and returns a string
def remove_stopwords(string, extra_words = [], exclude_words = []):
    stopword_list = stopwords.words('english')
    # use set casting to remove any excluded stopwords
    stopword_set = set(stopword_list) - set(exclude_words)
    # add in extra words to stopwords set using a union
    stopword_set = stopword_set.union(set(extra_words))
    # split the document by spaces
    words = string.split()
    # every word in our document that is not a stopword
    filtered_words = [word for word in words if word not in stopword_set]
    # join it back together with spaces
    string_without_stopwords = ' '.join(filtered_words)
    return string_without_stopwords

### 6. Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe `news_df`.

In [ ]:
news_df = acquire.get_news_articles_data()
news_df

### 7. Make another dataframe for the Codeup blog posts. Name the dataframe `codeup_df`.

In [ ]:
codeup_df = acquire.get_blog_articles_data()
codeup_df

### 8. For each dataframe, produce the following columns:

* `title` to hold the title
* `original` to hold the original article/post content
* `clean` to hold the normalized and tokenized original with the stopwords removed.
* `stemmed` to hold the stemmed version of the cleaned data.
* `lemmatized` to hold the lemmatized version of the cleaned data.

In [ ]:
# Work flow:
#  1. clean: normalized/tokenized, with stopwords removed apply: basic_clean, tokenize, remove_stopwords

#  2. stemmed: stemmed version of cleaned data apply: stem function onto cleaned data

#  3. lemmatized: lemmatized version of cleaned data apply: lemmatize function onto cleaned datas

In [ ]:
news_df.rename(columns={'content': 'original'}, inplace=True)
codeup_df.rename(columns={'content': 'original'}, inplace=True)

In [ ]:
news_df

In [ ]:
codeup_df

In [ ]:
# This function takes in a df and the string name for a text column with option to pass lists for extra_words and exclude_words
# and returns a df with the text article title, original text, stemmmed text, lemmatized text, cleaned, tokenized, & lemmatized 
# text with stopwords removed
def prep_article_data(df, column, extra_words=[], exclude_words=[]):
    df['clean'] = df[column].apply(basic_clean)\
                            .apply(tokenize)\
                            .apply(remove_stopwords,
                                  extra_words=extra_words,
                                  exclude_words=exclude_words)
    
    df['stemmed'] = df['clean'].apply(stem)
    
    df['lemmatized'] = df['clean'].apply(lemmatize)
    
    return df[['title', column,'clean', 'stemmed', 'lemmatized']]

In [ ]:
prep_article_data(news_df, 'original', extra_words = ['ha'], exclude_words = ['no'])

In [ ]:
prep_article_data(codeup_df, 'original', extra_words = ['ha'], exclude_words = ['no']).head()

### 9. Ask yourself:

* If your corpus is 493KB, would you prefer to use stemmed or lemmatized text?
* If your corpus is 25MB, would you prefer to use stemmed or lemmatized text?
* If your corpus is 200TB of text and you're charged by the megabyte for your hosted computational resources, would you prefer to use stemmed or lemmatized text?